In [3]:
from datasets import load_dataset
dataset = load_dataset("tilde_model", lang1="de", lang2="is")

Using custom data configuration de-is-lang1=de,lang2=is


Downloading:   0%|          | 0.00/22.4M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

Dataset tilde_model downloaded and prepared to C:\Users\oddur\.cache\huggingface\datasets\tilde_model\de-is-lang1=de,lang2=is\0.0.0\cea5250cab9d7f20be33fd2b08330b2e17bf4de020ded2719dda9b288e99a205. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [22]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 399681
    })
})


In [25]:
dataset_tab = load_dataset('text', data_files='../data/raw_data/Tatoeba_de-is.tsv')

Using custom data configuration default-d7978e86c955ce53


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset text downloaded and prepared to C:\Users\oddur\.cache\huggingface\datasets\text\default-d7978e86c955ce53\0.0.0\e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
print(dataset_tab)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1013
    })
})


In [34]:
#laga gögn til að þetta se´ekki weird
print(dataset_tab['train'][1])

{'text': '284\tIch spreche kein Japanisch.\t399287\tÉg tala ekki japönsku.'}
